In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from cardinality_estimation.featurizer import Featurizer
from query_representation.query import load_qrep

import glob
import random
import os
import json
import time
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
TRAINDIR = os.path.join(os.path.join("../", "queries"), "synth_2d_gaussian100K")
TRAINDIR2 = os.path.join(os.path.join("../", "queries"), "synth_2d_gaussian10M")

In [ ]:

def load_qdata(fns):
    qreps = []
    for qfn in fns:
        qrep = load_qrep(qfn)
        # TODO: can do checks like no queries with zero cardinalities etc.
        qreps.append(qrep)
        template_name = os.path.basename(os.path.dirname(qfn))
        qrep["name"] = os.path.basename(qfn)
        qrep["template_name"] = template_name
    return qreps

def get_query_fns(basedir, template_fraction=1.0):
    fns = []
    tmpnames = list(glob.glob(os.path.join(basedir, "*")))
    assert template_fraction <= 1.0
    
    for qi,qdir in enumerate(tmpnames):
        if os.path.isfile(qdir):
            continue
        template_name = os.path.basename(qdir)
        # let's first select all the qfns we are going to load
        qfns = list(glob.glob(os.path.join(qdir, "*.pkl")))
        qfns.sort()
        num_samples = max(int(len(qfns)*template_fraction), 1)
        random.seed(1234)
        qfns = random.sample(qfns, num_samples)
        fns += qfns
    return fns

In [ ]:
train_qfns = get_query_fns(TRAINDIR, template_fraction = 1.0)
train_qfns2 = get_query_fns(TRAINDIR2, template_fraction = 1.0)
trainqs = load_qdata(train_qfns)
trainqs2 = load_qdata(train_qfns2)

In [ ]:
allcards = []
allcards2 = []
for query in trainqs:
    for node in query["subset_graph"].nodes():
        cards = query["subset_graph"].nodes()[node]["cardinality"]
        if "actual" not in cards:
            continue
        y = cards["actual"]
        allcards.append(y)
        
for query in trainqs2:
    for node in query["subset_graph"].nodes():
        cards = query["subset_graph"].nodes()[node]["cardinality"]
        if "actual" not in cards:
            continue
        y = cards["actual"]
        allcards2.append(y)

In [ ]:
sns.distplot(allcards, kde=False)

In [ ]:
sns.distplot(allcards2, kde=False)